In [18]:
# Specify the file path
file_path5 = r'C:\Users\Code file-Hill_climbing\Stock portfolio selection\Final_Input_dataset.csv'

# Read the csv file into a Pandas DataFrame
dataset = pd.read_csv(file_path5)

C:\Users\Dell\AppData\Local\Temp\ipykernel_12784\2216304090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[['normalized_fitness_score', 'normalized_percent_change_from_instrinsic_value', 'normalized_Rev_Gr_Next_Y']] = dataset[['normalized_fitness_score', 'normalized_percent_change_from_instrinsic_value', 'normalized_Rev_Gr_Next_Y']].fillna(0)


In [4]:
import numpy as np
import pandas as pd

class StockData:
    def __init__(self, file_path5):
        # Load data from CSV file
        self.dataset = pd.read_csv(file_path5)

    def get_data(self):
        return self.dataset.values.tolist()

# Creating an instance of StockData with the file path
stock_data = StockData(file_path5)

# Getting the data 
list_data = stock_data.get_data()

# Printing the data
for item in list_data:
    print(item)

[1, 'AAPL', 'Apple', 197, 1, 2.61, 68.66458906, -188.3, 5.68, 2, 512, 0.403899721448468, 0.8147214193807021, 0.492261607588617]
[2, 'MSFT', 'Microsoft', 374, 1, 3.4, 128.2130242, -191.99, 13.93, 1, 514, 0.6239554317548748, 0.8123355252523293, 0.5746380429355965]
[3, 'GOOGL', 'Alphabet', 133, 1, 3.68, 107.8190101, -24.25, 11.32, 2, 512, 0.7019498607242342, 0.920793487608222, 0.5485771342985521]
[4, 'AMZN', 'Amazon.com', 148, 1, 3.03, 36.10092106, -312.29, 11.35, 1, 511, 0.520891364902507, 0.7345514971647301, 0.5488766849725412]
[5, 'NVDA', 'NVIDIA', 480, 1, 3.93, 1038.101036, 53.68, 56.53, 1, 511, 0.7715877437325906, 0.9711817611648854, 1.0]
[6, 'META', 'Meta Platforms', 334, 1, 3.29, 275.130394, -21.67, 13.08, 1, 511, 0.5933147632311978, 0.9224616737467588, 0.5661507738392411]
[7, 'BRK.B', 'Berkshire Hathaway', 362, 1, 3.06, 127.7294842, -183.43, 0.97, 2, 512, 0.5292479108635099, 0.8178702823631343, 0.4452321517723414]
[8, 'TSLA', 'Tesla', 239, 1, 3.11, 17.99574905, -1229.7, 20.81, 1, 

In [5]:
class BudgetAllocation:
    def __init__(self, selected_stocks, total_budget):
        self.selected_stocks = selected_stocks
        self.total_budget = total_budget

    def calculate_budget_allocation(self, stock_price, min_quantity, max_quantity):
        upper_budget = int(self.total_budget * 0.20)
        lower_budget = int(self.total_budget * 0.05)

        for quantity in range(min_quantity, max_quantity + 1):
            budget_used = stock_price * quantity
            if lower_budget <= budget_used <= upper_budget:
                return quantity, budget_used, self.total_budget - budget_used, True
        return None, None, None, False

# Instantiate the BudgetAllocation class 
#budget_allocator = BudgetAllocation(selected_stocks, total_budget)

In [18]:
import random

# Get user input for cardinality and budget
cardinality = int(input("Please enter the cardinality for portfolio : "))
total_budget = int(input("Enter the total budget limit : "))

# Category constraint inputs
large_cap_count = int(input("Enter the number of large-cap stocks to select : "))
mid_cap_count = int(input("Enter the number of mid-cap stocks to select : "))
small_cap_count = cardinality - large_cap_count - mid_cap_count


# Function to ensure category constraint is respected
def check_category_constraint(selected_stocks, large_cap_count, mid_cap_count, small_cap_count):
    large_cap = sum(1 for stock in selected_stocks if stock[4] == 1)
    mid_cap = sum(1 for stock in selected_stocks if stock[4] == 2)
    small_cap = sum(1 for stock in selected_stocks if stock[4] == 3)
    return large_cap == large_cap_count and mid_cap == mid_cap_count and small_cap == small_cap_count


# Check_constraints to ensure all constraints are satisfied
def check_constraints(selected_stocks, budget_allocator, large_cap_count, mid_cap_count, small_cap_count):
    if len(selected_stocks) != cardinality:
        return False, 0, []  # Added empty list to track quantities

    if not check_category_constraint(selected_stocks, large_cap_count, mid_cap_count, small_cap_count):
        return False, 0, []

    total_budget_used = 0
    quantities = []  # List to store quantity for each stock
    for stock in selected_stocks:
        stock_price = stock[3]
        min_quantity = stock[9]
        max_quantity = stock[10]

        # Calculate budget allocation for each stock
        quantity, budget_used, _, success = budget_allocator.calculate_budget_allocation(stock_price, min_quantity, max_quantity)
        if not success:
            return False, 0, []
        total_budget_used += budget_used
        quantities.append(quantity)

    if total_budget_used > budget_allocator.total_budget:
        return False, 0, []

    return True, total_budget_used, quantities


# Generate a feasible solution
while True:
    selected_stocks = random.sample(list_data, cardinality)  # Initial random selection
    budget_allocator = BudgetAllocation(selected_stocks, total_budget)

    # Check if this solution meets all constraints
    is_feasible, total_budget_used, quantities = check_constraints(
        selected_stocks, budget_allocator, large_cap_count, mid_cap_count, small_cap_count
    )
    if is_feasible:
        break  # Stop once a feasible solution is found

# Prepare selected stocks output with only specific columns
selected_stocks = [
    [stock[0], stock[1], stock[2], stock[3], stock[4], quantities[i]] for i, stock in enumerate(selected_stocks)
]

# Print selected stocks and total budget used
print("\nInitial Portfolio Selected Stocks:")
for stock in selected_stocks:
    print(stock)

print(f"Total Budget Used: {total_budget_used}")

budget_allocator = BudgetAllocation(selected_stocks, total_budget)


Please enter the cardinality for portfolio :  12
Enter the total budget limit :  30000
Enter the number of large-cap stocks to select :  6
Enter the number of mid-cap stocks to select :  3



Initial Portfolio Selected Stocks:
[52, 'WFC', 'Wells Fargo & Company', 53, 1, 34]
[433, 'MOS', 'The Mosaic Company', 51, 3, 36]
[171, 'WMG', 'Warner Music Group', 53, 3, 34]
[199, 'KLAC', 'KLA Corporation', 699, 1, 3]
[440, 'THO', 'Thor Industries, Inc.', 105, 3, 18]
[33, 'PEP', 'PepsiCo', 172, 1, 11]
[196, 'ADI', 'Analog Devices, Inc.', 195, 1, 10]
[242, 'MCK', 'McKesson Corporation', 527, 1, 4]
[260, 'AON', 'Aon plc', 316, 1, 6]
[131, 'KVUE', 'Kenvue', 51, 2, 36]
[108, 'AEP', 'American Electric Power Company', 83, 2, 22]
[159, 'MKC', 'McCormick & Company', 69, 2, 27]
Total Budget Used: 22798


In [19]:
import pandas as pd

class Solution:
    def normalize_total_budget(value, new_min, new_max):
        min_range = 0
        max_range = total_budget

        # Normalize the value within the specified range
        normalized_budget_value = ((value - min_range) / (max_range - min_range)) * (new_max - new_min) + new_min

        return normalized_budget_value

    #def calculate_objective_function(reallocated_stocks, file_path5, normalized_total_budget):
    def calculate_objective_function(selected_stocks, file_path5, normalized_total_budget):
        weight_fitness = 0.5
        weight_percent_change = 0.2
        weight_rev_growth = 0.25
        weight_normalized_budget = 0.05

        # Read the data from the new file
        new_data = pd.read_csv('Normalised_value_dataset.csv')

        total_objective_function = 0

        #for stock in reallocated_stocks:
        for stock in selected_stocks:
            stock_id = stock[0]

            # Fetching data for the current stock from the new data file based on the stock ID
            stock_data = new_data[new_data['Stock_ID'] == stock_id]
            if not stock_data.empty:
                normalized_fitness_score = stock_data['normalized_fitness_score'].values[0]
                normalized_percent_change = stock_data['normalized_percent_change_from_instrinsic_value'].values[0]
                normalized_rev_growth = stock_data['normalized_Rev_Gr_Next_Y'].values[0]
                
                # Calculating the objective function value for a given stock
                objective_function_value = (
                    weight_fitness * normalized_fitness_score +
                    weight_percent_change * normalized_percent_change +
                    weight_rev_growth * normalized_rev_growth +
                    weight_normalized_budget * normalized_total_budget
                )

                total_objective_function += objective_function_value

        return total_objective_function


# Create an instance of the Solution class
portfolio_solution = Solution()

# Normalizing total budget value
class_output_budget = total_budget_used
normalized_budget_value = Solution.normalize_total_budget(class_output_budget, 0, 1)  # Normalize within (0, 1) range

rounded_normalized_total_budget_used = round(normalized_budget_value, 2)

normalized_total_budget = rounded_normalized_total_budget_used

# Calculating objective function
#objective_function = Solution.calculate_objective_function(reallocated_stocks, file_path5, normalized_total_budget)
objective_function = Solution.calculate_objective_function(selected_stocks, file_path5, normalized_total_budget)
rounded_objective_function = round(objective_function, 4)

# Print the selected stocks as a list of lists
print("Selected Stocks:")
#for stock in reallocated_stocks[:]:
for stock in selected_stocks[:]:
    print(stock)

# Print the total budget used
print(f"Total Budget Used: {total_budget_used}")

# Print the normalised total budget used
print("Normalized Total Budget Used: ", rounded_normalized_total_budget_used)

# Print the total objective function value
print(f"Total Objective Function Value: {rounded_objective_function}")

Selected Stocks:
[52, 'WFC', 'Wells Fargo & Company', 53, 1, 34]
[433, 'MOS', 'The Mosaic Company', 51, 3, 36]
[171, 'WMG', 'Warner Music Group', 53, 3, 34]
[199, 'KLAC', 'KLA Corporation', 699, 1, 3]
[440, 'THO', 'Thor Industries, Inc.', 105, 3, 18]
[33, 'PEP', 'PepsiCo', 172, 1, 11]
[196, 'ADI', 'Analog Devices, Inc.', 195, 1, 10]
[242, 'MCK', 'McKesson Corporation', 527, 1, 4]
[260, 'AON', 'Aon plc', 316, 1, 6]
[131, 'KVUE', 'Kenvue', 51, 2, 36]
[108, 'AEP', 'American Electric Power Company', 83, 2, 22]
[159, 'MKC', 'McCormick & Company', 69, 2, 27]
Total Budget Used: 22798
Normalized Total Budget Used:  0.76
Total Objective Function Value: 5.8363


In [20]:
current_portfolio = selected_stocks[:]

In [21]:
current_portfolio

[[52, 'WFC', 'Wells Fargo & Company', 53, 1, 34],
 [433, 'MOS', 'The Mosaic Company', 51, 3, 36],
 [171, 'WMG', 'Warner Music Group', 53, 3, 34],
 [199, 'KLAC', 'KLA Corporation', 699, 1, 3],
 [440, 'THO', 'Thor Industries, Inc.', 105, 3, 18],
 [33, 'PEP', 'PepsiCo', 172, 1, 11],
 [196, 'ADI', 'Analog Devices, Inc.', 195, 1, 10],
 [242, 'MCK', 'McKesson Corporation', 527, 1, 4],
 [260, 'AON', 'Aon plc', 316, 1, 6],
 [131, 'KVUE', 'Kenvue', 51, 2, 36],
 [108, 'AEP', 'American Electric Power Company', 83, 2, 22],
 [159, 'MKC', 'McCormick & Company', 69, 2, 27]]

In [22]:
import copy
import csv
import random


# Function to generate neighboring portfolios
def generate_neighbor_portfolio(current_portfolio, total_budget_used, total_budget):
    # Read the CSV file and store its contents in 'row'
    row = []
    with open('Normalised_value_dataset.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip the header row
        for r in reader:
            row.append(r)  # Store all columns for each row, assuming constraints might need more columns

    total_budget_used_5 = total_budget_used

    while True:  # Keep generating portfolios until a valid one is found
        neighbor_portfolio = copy.deepcopy(current_portfolio)

        index_to_swap_1 = random.randint(0, len(current_portfolio) - 1)
        index_to_swap_2 = random.randint(0, len(current_portfolio) - 1)

        while index_to_swap_2 == index_to_swap_1:
            index_to_swap_2 = random.randint(0, len(current_portfolio) - 1)

        reduced_total_budget = total_budget_used_5 - (current_portfolio[index_to_swap_1][3] * current_portfolio[index_to_swap_1][5] + 
                                                      current_portfolio[index_to_swap_2][3] * current_portfolio[index_to_swap_2][5])
        
        
       

        # Pre-filter rows that satisfy upper and lower budget constraints for efficiency
        filtered_rows_1_options = [
            r for r in row if r[4] == str(int(current_portfolio[index_to_swap_1][4]))
            and int(r[3]) * int(r[9]) <= int(total_budget * 0.20)
            and int(r[3]) * int(r[10]) >= int(total_budget * 0.05)
        ]
        
        filtered_rows_2_options = [
            r for r in row if r[4] == str(int(current_portfolio[index_to_swap_2][4]))
            and int(r[3]) * int(r[9]) <= int(total_budget * 0.20)
            and int(r[3]) * int(r[10]) >= int(total_budget * 0.05)
        ]
        
        retry_limit = 1000  # Set a limit to retries
        retry_count = 0
        
        while retry_count < retry_limit:
            retry_count += 1
            
            # Select rows based on the category to maintain a balanced portfolio
            filtered_rows_1 = random.choice(filtered_rows_1_options)
            filtered_rows_2 = random.choice(filtered_rows_2_options)
        
            current_portfolio_ids = {int(item[0]) for item in current_portfolio}

            # Ensure no duplicates from the current portfolio
            if int(filtered_rows_1[0]) in current_portfolio_ids or int(filtered_rows_2[0]) in current_portfolio_ids:
                continue
        
            # Define upper and lower budget limits
            upper_budget = int(total_budget * 0.20)
            lower_budget = int(total_budget * 0.05)
        
            # Calculate feasible quantity ranges based on budget constraints
            max_quantity_1 = min(int(filtered_rows_1[10]), upper_budget // int(filtered_rows_1[3]))
            min_quantity_1 = max(int(filtered_rows_1[9]), (lower_budget + int(filtered_rows_1[3]) - 1) // int(filtered_rows_1[3]))
            max_quantity_2 = min(int(filtered_rows_2[10]), upper_budget // int(filtered_rows_2[3]))
            min_quantity_2 = max(int(filtered_rows_2[9]), (lower_budget + int(filtered_rows_2[3]) - 1) // int(filtered_rows_2[3]))
        
            # Check if the feasible ranges are valid
            if min_quantity_1 > max_quantity_1 or min_quantity_2 > max_quantity_2:
                continue
        
            # Generate random quantities within the feasible limits
            random_limit_1 = random.randint(min_quantity_1, max_quantity_1)
            random_limit_2 = random.randint(min_quantity_2, max_quantity_2)
        
            # Calculate budgets for these quantities
            random_limit_1_budget = random_limit_1 * int(filtered_rows_1[3])
            random_limit_2_budget = random_limit_2 * int(filtered_rows_2[3])

            # Check if both budgets meet the constraints
            if (lower_budget <= random_limit_1_budget <= upper_budget and
                lower_budget <= random_limit_2_budget <= upper_budget and
                reduced_total_budget + random_limit_1_budget + random_limit_2_budget <= total_budget):
        
                # Update quantities in the selected rows
                filtered_rows_1[5] = float(random_limit_1)
                filtered_rows_2[5] = float(random_limit_2)
        
                # Replace stocks in the neighbor portfolio
                neighbor_portfolio[index_to_swap_1] = filtered_rows_1
                neighbor_portfolio[index_to_swap_2] = filtered_rows_2
                break  # Break when a feasible portfolio is generated
        
      
      
        
        # Convert strings to floats in the neighbor portfolio
        column_indices_to_keep = [1, 2] # Keep columns 1 and 2 as they are
        
        neighbor_portfolio = [
            [
                float(val) if idx not in column_indices_to_keep and val not in ['', None] else val
                for idx, val in enumerate(sublist)
            ]
            for sublist in neighbor_portfolio
        ]


        # If constraints are satisfied, calculate the budget and objective function
        total_budget_used_5 = sum(row[3] * row[5] for row in neighbor_portfolio)

        # Normalize total budget value
        normalized_budget_value_2 = Solution.normalize_total_budget(total_budget_used_5, 0, 1)  # Normalize within (0, 1) range

        rounded_normalized_total_budget_used_2 = round(normalized_budget_value_2, 2)

        normalized_total_budget_2 = rounded_normalized_total_budget_used_2

        # Calculate objective function value
        objective_function_1 = Solution.calculate_objective_function(neighbor_portfolio, file_path5, normalized_total_budget_2)
        rounded_objective_function_1 = round(objective_function_1, 4)
        total_objective_function_value = rounded_objective_function_1

        return neighbor_portfolio, total_budget_used_5, normalized_total_budget_2, total_objective_function_value

           

In [23]:
# Generate a neighbor portfolio using the 'row' read from the CSV file
neighbor_portfolio = generate_neighbor_portfolio(current_portfolio, total_budget_used, total_budget)

# Print the neighbor portfolio
print("Neighbor Portfolio:")
for row in neighbor_portfolio[0]:
    print(row[:6])

# Printing the final calculated value
print("Total Budget Used:", neighbor_portfolio[1])

# Printing the final calculated value
print("Normalised Total Budget Used:", neighbor_portfolio[2])

# Printing the final calculated value
print("Objective Function Value:", neighbor_portfolio[3])

Neighbor Portfolio:
[52.0, 'WFC', 'Wells Fargo & Company', 53.0, 1.0, 34.0]
[433.0, 'MOS', 'The Mosaic Company', 51.0, 3.0, 36.0]
[171.0, 'WMG', 'Warner Music Group', 53.0, 3.0, 34.0]
[199.0, 'KLAC', 'KLA Corporation', 699.0, 1.0, 3.0]
[369.0, 'LW', 'Lamb Weston Holdings, Inc.', 101.0, 3.0, 47.0]
[33.0, 'PEP', 'PepsiCo', 172.0, 1.0, 11.0]
[196.0, 'ADI', 'Analog Devices, Inc.', 195.0, 1.0, 10.0]
[242.0, 'MCK', 'McKesson Corporation', 527.0, 1.0, 4.0]
[260.0, 'AON', 'Aon plc', 316.0, 1.0, 6.0]
[307.0, 'BIIB', 'Biogen Inc.', 223.0, 2.0, 19.0]
[108.0, 'AEP', 'American Electric Power Company', 83.0, 2.0, 22.0]
[159.0, 'MKC', 'McCormick & Company', 69.0, 2.0, 27.0]
Total Budget Used: 28056.0
Normalised Total Budget Used: 0.94
Objective Function Value: 6.1029


In [24]:
import random
import copy
import csv
import time
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

# Start timer before the optimization process begins
start_time = time.time()


def hill_climbing_algorithm(current_portfolio):
    best_portfolio = current_portfolio
    best_objective_value = rounded_objective_function
    best_total_budget_used = total_budget_used

    objective_values_over_iterations = []  # List to store objective values at each iteration


    for i in range(5000):
        neighbor_portfolio, total_budget_used_5, normalized_total_budget_2, total_objective_function_value = generate_neighbor_portfolio(best_portfolio,best_total_budget_used, total_budget)

        # Comparison logic for objective function values
        if total_objective_function_value > best_objective_value:
            best_portfolio = neighbor_portfolio
            best_total_budget_used = total_budget_used_5
            best_normalized_total_budget = normalized_total_budget_2
            best_objective_value = total_objective_function_value
            
        # Append the objective value at each iteration to the list
        objective_values_over_iterations.append(total_objective_function_value)
            
    return best_portfolio, best_total_budget_used, best_normalized_total_budget, best_objective_value, objective_values_over_iterations



# Running hill climbing algorithm
best_portfolio, best_total_budget_used, best_normalized_total_budget, best_objective_value, objective_values_over_iterations = hill_climbing_algorithm(current_portfolio)


# End timer after the optimization process completes
end_time = time.time()

# Calculate the elapsed time
time_taken = end_time - start_time


# Printing the best portfolio found
print("Best Portfolio Found:")
for row in best_portfolio:
    print(row[:6])

# Printing the best_total_budget_used and best objective value
print("Total Budget Used:", best_total_budget_used)

print("Normalised Total Budget Used:", best_normalized_total_budget)

print("Best Objective Function Value:", best_objective_value)

# Print or log the time taken
print(f"Time taken by the hill climbing algorithm: {time_taken:.2f} seconds")


Best Portfolio Found:
[13.0, 'TSM', 'Taiwan Semiconductor Manufacturing Company', 102.0, 1.0, 17.0]
[402.0, 'MEDP', 'Medpace Holdings, Inc.', 405.0, 3.0, 11.0]
[380.0, 'NBIX', 'Neurocrine Biosciences, Inc.', 139.0, 3.0, 15.0]
[275.0, 'PXD', 'Pioneer Natural Resources Company', 243.0, 1.0, 8.0]
[410.0, 'REXR', 'Rexford Industrial Realty, Inc.', 53.0, 3.0, 29.0]
[5.0, 'NVDA', 'NVIDIA', 480.0, 1.0, 7.0]
[60.0, 'HDB', 'HDFC Bank', 65.0, 1.0, 35.0]
[44.0, 'PDD', 'PDD Holdings', 146.0, 1.0, 18.0]
[203.0, 'FI', 'Fiserv, Inc.', 151.0, 1.0, 10.0]
[306.0, 'FANG', 'Diamondback Energy, Inc.', 183.0, 2.0, 18.0]
[148.0, 'YPF', 'YPF SA', 57.0, 2.0, 30.0]
[290.0, 'MPWR', 'Monolithic Power Systems, Inc.', 732.0, 2.0, 3.0]
Total Budget Used: 28728.0
Normalised Total Budget Used: 0.96
Best Objective Function Value: 8.5614
Time taken by the hill climbing algorithm: 66.65 seconds
